In [1]:
import numpy as np
import pandas as pd

import torch

from data_management import fetch_dataset

In [2]:
from data_management import fetch_dataset, consistent_subset

In [3]:
%run dataset_files/metadata.py

In [4]:
len(corruption_names_train)

15

### Dataset creation

#### Time to run calculations:

In hours.

In [63]:
num_samples = 600
5 * num_samples *15 / 60 / 60

12.5

Wilson 95% confidence intervals (not perfect, but just for sample size selection)

In [70]:
from scipy.stats import norm

z= norm.ppf(1-0.05/2)
p_gt = 0.5

print( z * ( p_gt * (1-p_gt) /num_samples)**0.5)

0.04000759730296092


Honestly, the scaling is pretty bad. For this initial reproduction, I think I'm going to go with 600 samples for the sake of speed.

In [5]:
for cname in corruption_names_train:
    dataset = fetch_dataset(f'dataset_files/corrupted_{cname}_severity5_valid.npz')
    subset = consistent_subset(dataset, count=600)
    torch.save(subset, f'data_subsets/corrupted_{cname}.pt')

## Run adaptation:

In [5]:
from TTA_experiments import TTA_curve

pd.DataFrame(columns=['model_layers', 'corruption']+[f'step{5*i}' for i in range(7)]
            ).to_csv('results.csv', index=False)

W0917 00:02:46.206000 26994 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


import error: No module named 'triton'


In [6]:
for cname in corruption_names_train:
    data = torch.load(f'data_subsets/corrupted_{cname}.pt', weights_only=False)
    results = TTA_curve(2, data, device='mps:0')
    results_dict = {f'step{5*i}': results[i] for i in range(7)}
    row_dict = {'model_layers': 2, 'corruption': cname, **results_dict}
    
    pd.DataFrame([row_dict]).to_csv('results.csv', mode='a', header=False, index=False)

In [7]:
pd.read_csv('results.csv')

,model_layers,corruption,step0,step5,step10,step15,step20,step25,step30
0,2,gaussian_noise,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2,shot_noise,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,impulse_noise,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2,defocus_blur,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333
4,2,glass_blur,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333
5,2,motion_blur,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333
6,2,zoom_blur,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333
7,2,snow,0.000000,0.000000,0.000000,0.000000,0.000000,0.001667,0.001667
8,2,frost,0.000000,0.001667,0.001667,0.001667,0.001667,0.003333,0.003333
9,2,fog,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
